In [1]:
import os
import numpy as np
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import cv2

def process_dataset(root_folder):
    image_paths = []
    label_data = []

    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]

                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)

                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data.append(label)

    return image_paths, label_data


def resize_img(image_paths):
    images_resized = []  # 리사이즈된 이미지를 저장할 리스트
    for image_path in image_paths:
        image = cv2.imread(image_path)  # 각 이미지 경로로부터 이미지를 읽음
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 색상 변환
        image_resized = cv2.resize(image, (128, 128))  # 이미지 리사이즈
        images_resized.append(image_resized)  # 결과 리스트에 추가
    images_resized = np.array(images_resized) / 255.0  # numpy 배열로 변환 및 정규화
    return images_resized


def shuffle_data(image_paths, label_data):
    # 데이터와 레이블을 같이 섞어줍니다.
    indices = np.arange(len(image_paths))
    np.random.shuffle(indices)
    shuffled_image_paths = np.array(image_paths)[indices]
    shuffled_label_data = np.array(label_data)[indices]
    return shuffled_image_paths, shuffled_label_data


def multilabel_train_generator(image_paths, label_data, batch_size):
    num_samples = len(image_paths)
    while True:
        # 데이터 셔플
        image_paths, label_data = shuffle_data(image_paths, label_data)
        for offset in range(0, num_samples, batch_size):
            batch_images = []
            batch_labels = []

            # 배치 크기만큼 이미지와 레이블 데이터 로드 및 전처리
            batch_image_paths = image_paths[offset:offset + batch_size]
            batch_image_labels = label_data[offset:offset + batch_size]

            batch_images = resize_img(batch_image_paths)
            
            for labels in batch_image_labels:
                batch_labels.append(labels)

            # 배치 데이터 반환
            yield np.array(batch_images), np.array(batch_labels)


def multilabel_test_generator(image_paths, label_data, batch_size):
    num_samples = len(image_paths)
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_images = []
            batch_labels = []

            # 배치 크기만큼 이미지와 레이블 데이터 로드 및 전처리
            batch_image_paths = image_paths[offset:offset + batch_size]
            batch_image_labels = label_data[offset:offset + batch_size]

            batch_images = resize_img(batch_image_paths)
            
            for labels in batch_image_labels:
                batch_labels.append(labels)

            # 배치 데이터 반환
            yield np.array(batch_images), np.array(batch_labels)


7874 7874
1691 1691
1687 1687


In [ ]:
last_path = [[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,1,1]]
prefix = [f"{i:03d}" for i in range(505, 513)]
prefix_to_label = dict(zip(prefix, last_path))
# 각각의 데이터셋에 대해 함수를 호출
train_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
valid_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_folder = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

train_image_paths, train_label_data = process_dataset(train_folder)
valid_image_paths, valid_label_data = process_dataset(valid_folder)
test_image_paths, test_label_data = process_dataset(test_folder)
print(len(train_image_paths), len(train_label_data))

batch_size = 32

train_generator = multilabel_train_generator(train_image_paths, train_label_data, batch_size)
validation_generator = multilabel_test_generator(valid_image_paths, valid_label_data, batch_size)
test_generator = multilabel_test_generator(test_image_paths, test_label_data, batch_size)

# 모델 구성
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers:
    base_model.trainable = False
for layer in base_model.layers[-9:]:
    base_model.trainable = True

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(3, activation='sigmoid'))

earlystopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)


In [2]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.0002),
              loss=['binary_crossentropy'],
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=len(train_image_paths) // batch_size,
                    epochs=25,
                    validation_data=validation_generator,
                    validation_steps=len(valid_image_paths) // batch_size,
                    #callbacks=[earlystopping]
                    )

Epoch 1/25
246/246 [==============================] - 38s 125ms/step - loss: 0.6414 - accuracy: 0.2876 - val_loss: 0.7666 - val_accuracy: 0.1268
Epoch 2/25
246/246 [==============================] - 30s 120ms/step - loss: 0.5650 - accuracy: 0.3220 - val_loss: 0.7723 - val_accuracy: 0.1268
Epoch 3/25
246/246 [==============================] - 28s 112ms/step - loss: 0.5050 - accuracy: 0.3781 - val_loss: 0.7177 - val_accuracy: 0.1659
Epoch 4/25
246/246 [==============================] - 26s 108ms/step - loss: 0.4492 - accuracy: 0.4364 - val_loss: 0.6742 - val_accuracy: 0.1809
Epoch 5/25
246/246 [==============================] - 27s 111ms/step - loss: 0.3814 - accuracy: 0.4653 - val_loss: 0.5429 - val_accuracy: 0.4159
Epoch 6/25
246/246 [==============================] - 27s 111ms/step - loss: 0.3164 - accuracy: 0.4952 - val_loss: 0.6498 - val_accuracy: 0.4922
Epoch 7/25
246/246 [==============================] - 26s 107ms/step - loss: 0.2573 - accuracy: 0.5170 - val_loss: 0.7913 - val_ac

In [6]:
train_image_resized = resize_img(train_image_paths)
valid_image_resized = resize_img(valid_image_paths)
test_image_resized = resize_img(test_image_paths)
# 모델 예측
predictions = model.predict(test_image_resized)

# 임계값 설정 (예: 0.5)
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int)

# 각 레이블에 대한 정확도 계산
accuracy_per_label = np.mean(predictions_binary == test_label_data, axis=0)

# 각 레이블별 정확도 출력
for i, accuracy in enumerate(accuracy_per_label):
    print(f"레이블 {i}의 정확도: {accuracy}")

# 전체 정확도도 여전히 중요할 수 있으므로, 이를 계산합니다.
overall_accuracy = np.mean(predictions_binary == test_label_data)
print(f"전체 정확도: {overall_accuracy}")

53/53 [==============================] - 1s 24ms/step
레이블 0의 정확도: 0.7949021932424422
레이블 1의 정확도: 0.7931238885595732
레이블 2의 정확도: 0.8772969768820391
전체 정확도: 0.8217743528946848


In [11]:
# 테스트 데이터셋의 전체 샘플 수 계산
test_steps = np.ceil(len(test_image_paths) / batch_size)

predictions = model.predict(test_generator, steps=test_steps)

# 임계값 설정
threshold = 0.5
predictions_binary = (predictions > threshold).astype(int)

# 각 레이블에 대한 정확도 계산
accuracy_per_label = np.mean(predictions_binary == test_label_data, axis=0)

# 각 레이블별 정확도 출력
for i, accuracy in enumerate(accuracy_per_label):
    print(f"레이블 {i}의 정확도: {accuracy}")

# 전체 정확도도 여전히 중요할 수 있으므로, 이를 계산합니다.
overall_accuracy = np.mean(predictions_binary == test_label_data)
print(f"전체 정확도: {overall_accuracy}")

#model.save(r'E:\AImodel\models\Face-KneePushup-multiLabel-model')

53/53 [==============================] - 2s 34ms/step
레이블 0의 정확도: 0.44991108476585656
레이블 1의 정확도: 0.3959691760521636
레이블 2의 정확도: 0.5726141078838174
전체 정확도: 0.47283145623394585
